## BENDERS DECOMPOSITION METHOD
### Prepared by Bikey SERANILLA

#### Master Model

$$\begin{align}\min\ & 100x_1 + 150x_2 V \\
\end{align}$$

$$\begin{align}s.t. \\
& x_1 + x_2 \leq 120 \\
& x_1 \geq 40 \\
& x_2 \geq 20 \\
& V \geq 0 \\
&  V \geq (b-F^y)^Tu \\
& V \geq 0, y \in \mathbb{Z}\\
\end{align}$$ 

#### Subroblem Model

$$\begin{align}\min\ & E (q_1y_1 + q_2y_2)\\
\end{align}$$
$$\begin{align}s.t. \\
& 6y_1 + 10y_2 \leq 60x_1 \\
& 8y_1 + 5y_2 \leq 80x_1 \\
& y_1 \leq d_1 \\
& y_2 \leq d_2 \\
& y_1, y_2 \geq 0\\
\end{align}$$ 

$$\begin{align}
\xi = [[500, 100, -24, -28]& p_1=0.4\\
[300, 300, -28, -32]]& p_2=0.6\\
\end{align}$$

In [ ]:
from IPython.display import Image
Image(filename = "L-Shaped Algorithm.png", width = 400, height = 100)

In [18]:
import gurobipy as gp
from gurobipy import GRB
from numpy.random import randint, binomial
import numpy as np
from math import sqrt
from seaborn import distplot
from math import inf, isclose
import math as math

In [19]:
# Parameters
I = 2
objective_params_1 = np.array([100, 150])
W = np.array([[6, 10],
              [8, 5]])
RHS = np.array([[60],
              [80]])
SCENARIOS = np.array([[500, 100, -24, -28],
                      [300, 300, -28, -32]])

In [20]:
# MASTER PROBLEMS (MP)
# Master Problem 1 (initial MP)
masterProblem_1 = gp.Model()
masterProblem_1.Params.LogToConsole = 0
masterProblem_1.ModelSense = GRB.MINIMIZE
x = masterProblem_1.addVars(I, vtype=GRB.CONTINUOUS, lb=[40, 20], name='x')
masterProblem_1.addConstr(gp.quicksum(x[i] for i in range(I)) <= 120)
masterProblem_1.setObjective(gp.quicksum(objective_params_1[i]*x[i] for i in range(2)), GRB.MINIMIZE)

# Master Problem 2 (MP with cuts)
masterProblem_2 = gp.Model()
masterProblem_2.Params.LogToConsole = 0
masterProblem_2.ModelSense = GRB.MINIMIZE
x = masterProblem_2.addVars(I, vtype=GRB.CONTINUOUS, lb=[40, 20], name='x')
V = masterProblem_2.addVar(name='V', lb=-inf)
masterProblem_2.addConstr(gp.quicksum(x[i] for i in range(I)) <= 120)
masterProblem_2.setObjective(gp.quicksum(objective_params_1[i]*x[i] for i in range(2)) + V, GRB.MINIMIZE)

In [21]:
# SUBPROBLEM
def subProblem(xi, x):
    subProblem = gp.Model()
    subProblem.Params.LogToConsole = 0
    subProblem.ModelSense = GRB.MINIMIZE
    y = subProblem.addVars(I, vtype=GRB.CONTINUOUS, name='y')
    subProblem.addConstr(gp.quicksum(W[0][i]*y[i] for i in range(I)) <= RHS[0]*x[0], name="constr1")
    subProblem.addConstr(gp.quicksum(W[1][i]*y[i] for i in range(I)) <= RHS[1]*x[1], name="constr2")
    subProblem.addConstr(y[0] <= xi[0], name="constr3")
    subProblem.addConstr(y[1] <= xi[1], name="constr4")
    subProblem.setObjective(gp.quicksum(xi[2:][i]*y[i] for i in range(I)), GRB.MINIMIZE)
    subProblem.optimize()
    y_star = subProblem.getAttr('X', y)
    u_star = subProblem.Pi
    ObjVal = subProblem.ObjVal

    return y_star, u_star, ObjVal
    

In [22]:
# L-SHAPED DECOMPOSITION ALGORITHM

J = 10
UB = math.inf
LB = 0
w_0 = -inf
w = 0
count = 0

#Scenarios
S = 2
SCENARIOS = np.array([[500, 100, -24, -28],
                      [300, 300, -28, -32]])
h = np.array([[0, 0, 500, 100],
              [0, 0, 300, 300]])
T = np.array([[-60, 0, 0, 0],
              [0, -80, 0, 0]])
PROB = [0.4, 0.6]

#Second-stage Values
ssp_y = np.zeros((S,I))     #y_variables
ssp_u = np.zeros((S,4))     #duals / constraints
ssp_o = np.zeros((S))       #objective value

while(w != w_0):

    print("Iteration Number: ", count)

    if count == 0:
        #Step 1 : Solve first-stage problem
        FSP = masterProblem_1.optimize()
        FSP_ObjVal = masterProblem_1.ObjVal
        x_j = masterProblem_1.getAttr('X', x)
        print(x_j)
        print(FSP_ObjVal)
    else:
        #Step 1.2 : Add cut variable (V) and update objective function
        FSP = masterProblem_2.optimize()
        FSP_ObjVal = masterProblem_2.ObjVal
        x_j = masterProblem_2.getAttr('X', x)
        print(x_j)
        print(FSP_ObjVal)
    
    #Step 2: Solve each second-stage problem
    for s in range(S):
        #ssp = [y_star, w_star, u_star, ObjVal]
        ssp = subProblem(SCENARIOS[s], x_j)
        ssp_y[s] = ssp[0].values()
        ssp_u[s] = np.array(ssp[1])
        ssp_o[s] = ssp[2]
    # print("Duals = ", ssp_u)
    # print("Ys = ", ssp_y)
    # print("ObjVal = ", ssp_o)
    
    #Step 3: Generate optimality cuts
    
    # #Multi-cut approach
    # E = np.dot(ssp_u[:, :-1], SCENARIO.T)
    # e = np.dot(ssp_u.T, h)
    # for s in range(S):
    #     masterProblem.addConstr(V >= e[s] - gp.quicksum(E[s][i]*x[i] for i in range(CROPS)))

    # Single-cut approach
    E = sum(PROB[i]*np.dot(ssp_u,T.T)[i] for i in range(len(SCENARIOS)))
    e = sum(PROB[i]*np.dot(ssp_u[i].T,h[i]) for i in range(len(SCENARIOS)))
    masterProblem_2.addConstr(V >= e - gp.quicksum(E[i]*x[i] for i in range(len(SCENARIOS))))

    #Step 4: Stopping criterion
    w_0 = w
    w = e - sum(E[i]*x_j[i] for i in range(len(SCENARIOS)))
    print("w =", w)

    # Update bounds
    LB = max(LB, FSP_ObjVal)
    UB = min(UB, LB + np.mean(ssp_o))
    bound = UB - LB

    # masterProblem_2.addConstr(V >= w)
    
    count = count + 1

# A = masterProblem.ObjVal
# x_star = masterProblem_2.getAttr('X', x)
# x_star

Iteration Number:  0
{0: 40.0, 1: 20.0}
7000.0
w = -7470.4
Iteration Number:  1
{0: 40.0, 1: 80.0}
-2299.199999999997
w = -10032.0
Iteration Number:  2
{0: 66.82762201453792, 1: 53.172377985462084}
-1039.3748701973036
w = -14907.090342679126
Iteration Number:  3
{0: 40.0, 1: 33.74999999999999}
-889.4999999999982
w = -9744.0
Iteration Number:  4
{0: 46.66666666666665, 1: 36.24999999999999}
-855.8333333333303
w = -10959.999999999996
Iteration Number:  5
{0: 46.66666666666665, 1: 36.24999999999999}
-855.8333333333321
w = -10959.999999999996


In [ ]:
# L-SHAPED DECOMPOSITION ALGORITHM

J = 10
UB = math.inf
LB = 0
w_0 = -inf

#Scenarios
S = 2
SCENARIOS = np.array([[500, 100, -24, -28],
                      [300, 300, -28, -32]])
h = np.array([[0, 0, 500, 100],
              [0, 0, 300, 300]])
T = np.array([[-60, 0, 0, 0],
              [0, -80, 0, 0]])
PROB = [0.4, 0.6]

#Second-stage Values
ssp_y = np.zeros((S,I)) #y_variables
ssp_u = np.zeros((S,4))     #duals / constraints
ssp_o = np.zeros((S))       #objective value

for j in range(J):
    print("Iteration Number: ", j)

    if j == 0:
        #Step 1 : Solve first-stage problem
        FSP = masterProblem_1.optimize()
        FSP_ObjVal = masterProblem_1.ObjVal
        x_j = masterProblem_1.getAttr('X', x)
        print(x_j)
        print(FSP_ObjVal)
    else:
        #Step 1.2 : Add cut variable (V) and update objective function
        FSP = masterProblem_2.optimize()
        FSP_ObjVal = masterProblem_2.ObjVal
        x_j = masterProblem_2.getAttr('X', x)
        print(x_j)
        print(FSP_ObjVal)
    
    #Step 2: Solve each second-stage problem
    for s in range(S):
        #ssp = [y_star, w_star, u_star, ObjVal]
        ssp = subProblem(SCENARIOS[s], x_j)
        ssp_y[s] = ssp[0].values()
        ssp_u[s] = np.array(ssp[1])
        ssp_o[s] = ssp[2]
    # print("Duals = ", ssp_u)
    # print("Ys = ", ssp_y)
    # print("ObjVal = ", ssp_o)
    
    #Step 3: Generate optimality cuts
    
    # #Multi-cut approach
    # E = np.dot(ssp_u[:, :-1], SCENARIO.T)
    # e = np.dot(ssp_u.T, h)
    # for s in range(S):
    #     masterProblem.addConstr(V >= e[s] - gp.quicksum(E[s][i]*x[i] for i in range(CROPS)))

    # Single-cut approach
    # SCENARIOS_constraint = np.insert(SCENARIO, SCENARIO.shape[1], 0, axis=1)
    E = sum(PROB[i]*np.dot(ssp_u,T.T)[i] for i in range(len(SCENARIOS)))
    e = sum(PROB[i]*np.dot(ssp_u[i].T,h[i]) for i in range(len(SCENARIOS)))
    masterProblem_2.addConstr(V >= e - gp.quicksum(E[i]*x[i] for i in range(len(SCENARIOS))))
    
    # Update bounds
    LB = max(LB, FSP_ObjVal)
    UB = min(UB, LB + np.mean(ssp_o))
    bound = UB - LB

    j = j + 1

# A = masterProblem.ObjVal
x_star = masterProblem_2.getAttr('X', x)
x_star